## Modelling Mathematical Theorems
### Goals
* Determine if there is a trend among theorems of similar fields of Mathematics
* What are the most similar fields of Mathematics
* Build a predictive model to determine a field by commonly used words

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from urllib2 import Request, urlopen, HTTPError
from sklearn.neighbors import NearestNeighbors
from wordcloud import WordCloud, STOPWORDS
from nltk.stem.porter import PorterStemmer
from urlparse import urlunparse, urlparse
from nltk.corpus import wordnet as wn
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
import scipy.stats as stat
from nltk import corpus
import plasTeX as tex                                             # For Parsing TeX in Wikipedia 
import requests_cache
import pandas as pd
import scipy as sci
import numpy as np
import wordcloud
import nltk
import json 
import re

%matplotlib inline
plt.style.use('ggplot')
requests_cache.install_cache('coll_cache')
plt.rcParams.update({'font.size': 18})

### Scraping the Theorems

To get the Math Theorems, we scraped the Wikipedia page <a href="https://en.wikipedia.org/wiki/List_of_theorems">List of Theorems</a>. From here we recognized that there was a standard <tt>mw-content-ltr</tt> class in which the body of each page was located in. The following scapes the List of Theorems page, removes the header and footer, and then builds a data frame by scraping the pages of each of the theorems.

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_theorems'
html = urlopen(url)
soup = BeautifulSoup(html,'html.parser')
sections = soup.find_all('li')
sections = sections[37:1152] # Remove Header and Footer

In [ ]:
theorems = {'Field':[],'Title':[],'Link':[],'Page':[]}
for temp in sections:
    link = temp.find('a')
    cat = temp.find('i')
    if (cat == None):
        if (re.search(r'\(.*?\)',link.get('title')) == None):
            cat = ''
        else:
            cat = re.sub('\(|\)','',re.search(r'\(.*?\)',link.get('title')).group())
    else:
        cat = cat.get_text().lower()

    try:
        html = urlopen("https://en.wikipedia.org"+str(link.get('href')))
        page = BeautifulSoup(html,'html.parser')
    except HTTPError as e:  
        page = e
    
    if type(page) != HTTPError:
        theorems['Field'].append(cat)
        theorems['Title'].append(link.get('title'))
        theorems['Link'].append(link.get('href'))
        theorems['Page'].append(page.find(class_='mw-content-ltr'))
    
theorems = pd.DataFrame(theorems)
theorems.to_csv("theorems.txt",sep=',',encoding='utf-8') # Use encoding because of scraped pages
print theorems.shape

#### Data Pre-Processing

Once we had the html of each of the theorems, we needed to clean this text substantially in order analyze it. To do this we first replaced any unicode titles with their <tt>string</tt> equivalents, then we trimmed any theorems that were part of obscure fields of Mathematics (for example Lie Algebra, Metric Geometry, or Elliptic Differential Equations).

Still, we can't train the the html data as is. First, this data has all the html formatting tags such as <"p"> and <"div">. Beautiful Soup remove's these by the <tt>.get_text()</tt> function. Second, this is Mathematics and as such it contains a substantial amount of Math in each of the pages. Not only is this math not easily translated into words, it is formatted by the standard <tt>TeX</tt> formatting which looks like <tt>\forall x \in \mathbb{R}<\tt> to produce $\forall x \in \mathbb{R}$. To fix this we first removed the html formatting with get_text(), then used regular expressions to remove any brakets or other formatting that is present in a standard TeX script. However, this still left the above expression as <tt> forall x in mathbb R </tt>.

In [ ]:
## Clean up the table (get rid of Unicode titles)
for field in theorems['Field']:
    try:
        str(field)
    except UnicodeEncodeError as e:
        print field
theorems['Field'][['quantum theory' in x for x in theorems['Field']]] = 'quantum theory'
theorems['Field'][['clebsch' in x for x in theorems['Field']]] = 'clebsch gordan coefficients'
theorems['Field'] = [str(x) for x in theorems['Field']]

## Trim the data for analysis
trimmed = [" ".join([x for x in page.get_text().split() if re.search('\{*\}|\(*\)|\$*\$|\(|\)|\{|\}',x) == None]) for page in theorems['Page']]
theorems['Trimmed'] = trimmed

## Removes Theorems in Categories with only min_cnt entries
min_cnt = 4
good_fields = list(theorems.groupby('Field').count()[theorems.groupby('Field').count()['Link'] > min_cnt].index) 
duplicate_theorems = theorems[[(x in good_fields) for x in theorems['Field']]]
n = duplicate_theorems.shape[0]

To remove the extraneous words that were part of the TeX commands, we scraped a LaTeX glossary of commands and added these words to our collection of stopwords (used later for classification and word clouds). This glossary can be found <a href="https://en.wikibooks.org/wiki/LaTeX/Command_Glossary">here</a>.

In [ ]:
latexUrl = 'https://en.wikibooks.org/wiki/LaTeX/Command_Glossary'
try:
    latex_html = urlopen(latexUrl)
    soup = BeautifulSoup(latex_html,'html.parser')
except HTTPError as e:  
    print e
tex_words = []
bad_chars = r'[\\|\xc2|\x99|\xa0|{|}]'
for cat in soup.find_all('dl'):
    for word in soup.find_all('dt'):
        try:
            tex_words.append(str(re.sub(bad_chars,'',word.get_text()[1:])).lower())
        except UnicodeEncodeError as e:
            print word
        except IndexError as e:
            print word

stopwords = ['will','the','and','that','said',
             'from','they','their','this','year','ext',
             'mathbb','true','false','displaystyle','wikipedia',
             'ready','user','mediawiki','mw','value','theorem',
             'number','lemma','template','en','k_','cdot','x_',
             'pmod','styles','window','gadget','module','a_','rlq',
             'edit','system','varphi','mathbf','function','wikimedia',
             'scriptstyle','doi','references','mathrm']+tex_words

### Most Common Words

Next we used the nltk package to build word clouds and bar graphs of the most common words across all theorems in a given field of Mathematics. The following is a few functions used in this process.

In [ ]:
stemmer = PorterStemmer().stem
tokenize = nltk.word_tokenize

def remove_small(series,min_len = 3):
    """
        Removes any words less than or equal to min_len. Returns
        a block of text of all these words to be input into NLTK.
    """
    text = []
    [[text.append(y) for y in x.split()] for x in series]
    text = pd.DataFrame({'text':text})
    return(" ".join(text.iloc[[len(x)>min_len for x in text['text']]]['text']))

def stem(tokens,stemmer = PorterStemmer().stem):
    """
        Calls the nltk PorterStemmer() on a lowercase version 
        of a list of tokens. 
    """
    return [stemmer(w.lower()) for w in tokens] 

def lemmatize(text,stopwords=['']):
    """
    Extract simple lemmas based on tokenization and stemming
    Input: string
    Output: list of strings (lemmata)
    """
    #text = remove_small([text],min_len)
    return stem([x for x in tokenize(text) if x.lower() not in stopwords])

def build_wordcloud(text,stopwords,ax=plt,title=''):
    """
        Helper function for building a wordcloud with appropriate 
        title and formatting. Also returns the wordcloud as a 
        data frame of each word and its associated frequency.
    """
    wordcloud = WordCloud(stopwords = stopwords,font_path='/Library/Fonts/Apple Chancery.ttf').generate(text)
    ax.imshow(wordcloud)
    ax.axis("off")
    ax.show() if ax == plt else ''
    ax.set_title(title) if ax != plt else ''
    words = WordCloud(stopwords = stopwords).process_text(text)
    words = pd.DataFrame({'Words':words.keys(),'Freq':words.values()}).sort_values('Freq',ascending=0)    
    return(words)

def f(text,ax=plt,stopwords=[''],min_len=7,title=None): 
    """
       Helper function for build_wordcloud that adds the 
       title to the list of stopwords and removes and small
       words with remove_small first.
    """
    if (title == None):
        title = text.iloc[0,0]
    texts = [y.lower() for y in text['Trimmed']]
    words=build_wordcloud(stopwords=stopwords+title.split(' '),text=remove_small(texts,min_len),ax=ax,title=title)
    return(words)

We then built word clouds and bar graphs of the most common words in each field. 

In [ ]:
min_len = 6
grp = duplicate_theorems.groupby('Field')
fields = grp.apply(lambda x: x.iloc[0,0])
topwords = {}
for j in range(int(np.ceil(float(len(grp))/4.0))):
#for j in [8,10,13]: # Select theorems of interest
    f1,ax1 = plt.subplots(1,4,figsize = [20,10])
    f2,ax2 = plt.subplots(1,4,figsize = [20,3])
    for i in xrange(4):
        field = fields[i+4*j] if i+4*j < len(grp) else fields[-1]
        temp = duplicate_theorems[duplicate_theorems['Field']==field]
        txt=f(temp,ax1[i],stopwords=stopwords,min_len=min_len)
        txt[0:10].set_index('Words').plot(kind='bar',ax=ax2[i],title=field)
        topwords[field] = txt        

### Similarity of Fields

Next we measured the similarity of each field of Mathematics based on each fields most common words. The following builds a dictionary that provides a list of which fields of Math contain the Key in their most common words.

In [ ]:
textd = {}                                              # dictionary from lemmata to document ids containing that lemma
freqd = {}
min_freq = 5
for field in topwords.keys():
    freqd.update({field:sum(topwords[field]['Freq'][topwords[field]['Freq'] > min_freq].values)})
    t = " ".join(topwords[field]['Words'][topwords[field]['Freq'] > min_freq])    
    s = set(lemmatize(t,stopwords))
    try:
        toks = toks | s
    except NameError:
        toks = s
    for tok in s:
        try:
            textd[tok].append(field)
        except KeyError:
            textd[tok] = [field]

This dictionary is then turned into a data frame, below shows how many fields have the Keys as part of their top words. Below is a list of the ten most shared words among the fields.

In [ ]:
textd_save = textd
min_len = 4
textd = {key:vals for key,vals in textd_save.items() if len(key) > min_len}
print len(textd_save.keys()),len(textd.keys())
temp = pd.DataFrame({'Keys':textd.keys(), 'Length':[len(y) for y in textd.values()]}).sort_values('Length',ascending=False)
temp[0:10]

In [ ]:
def get_max_sim(C):
    """
        Returns the df of the most similar articles for a given corpus class. 
        Also returns a corpus for that df.
    """
    A = C
    temp = {'Max':[],'Argmax':[]}
    for i in range(A.shape[0]):
        temp['Argmax'].append((A.getcol(i)[range(i)+range(i+1,A.shape[0])]).A.argmax())
        temp['Max'].append((A.getcol(i)[range(i)+range(i+1,A.shape[0])]).max())
    df = pd.DataFrame(temp,index=range(A.shape[0])).sort_values(['Max'],ascending = 0)
    df['Field_1'] = np.array(fields)[df.index.values]
    df['Field_2'] = np.array(fields)[df['Argmax']]
    return(df)

def all_combos(seq):
    """
        Given a seq, returns all possible subsets of this sequence,
        with length equal to 2 and disregards order.
        Ex: all_combos([1,2,3]) = {[1,2],[1,3],[2,3]}
    """
    s = []
    for i in range(len(seq)):
        for j in range(i+1,len(seq)):
            s = s + [[seq[i],seq[j]]]
    return s

For $n$ total fields and $\nu$ total words, the similarity matrix for the fields of Mathematics, $Sim = [s_{ij}] \in \mathbb{R}^{n\times n}$, is given by,

$$s_{ij} = \frac{\sum_{k=1}^\nu \gamma_{ij}(w_k) }{\sqrt{2m_i m_j}};\quad\quad \gamma_{ij} = \left\lbrace
\begin{matrix} 
1&\text{if}&w_k\in F_i \cap F_j \\
0&\text{if}&i=j\\
0&\text{if}&w_k\notin F_i\cap F_j\
\end{matrix}\right.$$

where $F_i$ is the set of words for Field $i$, $w_k$ is the $k$th word, and $\beta$ is a scaling factor to force $s_{ij}\in [0,1]$. Below computes that similarity matrix and displays it as a graph of intensity. 

In [ ]:
n = max([len(x) for x in textd.values()])
combos = {}
for i in xrange(n+1):
    combos[i] = all_combos(xrange(i)) 
print {'Max{Fields per Word}:': n}

sim = np.zeros([len(fields),len(fields)])
field_key = {x[1]:x[0] for x in enumerate(fields)}

for key,val in textd.items():
    s = combos[len(val)]
    for indx in s:
        total_freq = np.sqrt(2*float(freqd[val[indx[0]]]+freqd[val[indx[1]]]))
        sim[field_key[val[indx[0]]],field_key[val[indx[1]]]] += float(1)/total_freq
        sim[field_key[val[indx[1]]],field_key[val[indx[0]]]] += float(1)/total_freq

import seaborn as sns

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sim = pd.DataFrame(sim).rename(index=str,columns={x:y for x,y in enumerate(fields)}).set_index(keys=fields)
sns.heatmap(sim, cmap=cmap, vmax=1, cbar = True,
            square=True, xticklabels=2, yticklabels=2,
            linewidths=.5)


max_sim = get_max_sim(sci.sparse.csr_matrix(sim))
max_sim[0:4]

### Classifying Field of Mathematics

Next we built a classifer to predict the field of Mathematics that a theorem belongs to. We tested using bag of words or just the top words of each field with both Multinomial Naive Bayes and SVM models. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
import random

def performance(true, pred,P=1):
    """
        Computes the True Positive, False Positive, True Negative, 
        and False Negative of a prediction.
    """
    TP = FP = TN = FN = 0
    for x,y in zip(true,pred): 
        if x==y==P:
           TP += 1
        if y==P and x!=y:
           FP += 1
        if y!=P and x==y:
           TN += 1
        if y!=P and x!=y:
           FN += 1

    return(TP, FP, TN, FN)

def test_clf(df,clf,topwords,top_x=15):
    """
        Tests a clf model for a given choice of using the top_x 
        most common words of a field.
    """
    if topwords != None:
        df.data = [" ".join(topwords[field]['Words'][0:top_x].values) for field in df.target_names]
    
    indx = range(df.shape[0])
    random.shuffle(indx)
    trainSz = int(np.floor(0.5*len(indx)))
    train_indx, test_indx = indx[:trainSz],indx[trainSz:]
    train_thm, test_thm = df.iloc[train_indx,:], df.iloc[test_indx,:]

    import sklearn.datasets
    train = sklearn.datasets.base.Bunch(
                    description=train_thm.description,
                    filenames=train_thm.filenames,
                    target_names=train_thm.target_names,
                    data=train_thm.data,
                    target=train_thm.target)

    test = sklearn.datasets.base.Bunch(
                    description=test_thm.description,
                    filenames=test_thm.filenames,
                    target_names=test_thm.target_names,
                    data=test_thm.data,
                    target=test_thm.target)

    clf = clf.fit(train.data, train.target)
    predicted = clf.predict(test.data)

    TP, FP, TN, FN = np.mean([performance(test.target, predicted,i) for i in set(predicted)],axis=0)
    acc = np.mean(predicted == test.target)    
    sens = TP/(TP+FN) if TP+FN > 0 else float('Inf')
    spec = TN/(FP+TN) if FP+TN > 0 else float('Inf')
    prec = TP/(TP+FP) if TP+FP > 0 else float('Inf')
    fall = FP/(FN+TN) if FN+TN > 0 else float('Inf')
    cm = {'Accuracy':acc,'Sensitivity':sens,
          'Specificity':spec,'Precision':prec,'Fallout':fall}
    return(cm)

NB  = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB()),
])
SVM = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42)),
])
new_duplicate = duplicate_theorems.iloc[range(37)+range(38,duplicate_theorems.shape[0]),:].copy()
new_duplicate['target'] = [field_key[x] for x in new_duplicate.Field]
new_duplicate.columns = ['target_names','filenames','Page','description','data','target']

#### Fitting Models to Bag of Words

Using a bag of words as a feature set we check both the Multinomial Naive Bayes (MNB) and SVM models. The SVM model fits multiple classes in 1 vs all fashion as it is a binary classifier. Hence it checks each Theorem against each field and returns the most likely field.

In [ ]:
temp = test_clf(new_duplicate,NB,None)
print 'MNB:',{x:round(y,4) for x,y in temp.items()}

svm_cm = test_clf(new_duplicate,SVM,None)
print 'SVM:',{x:round(y,4) for x,y in svm_cm.items()}

Since the accuracy was significantly lower than desired, but the SVM model was clearly better than the MNB model. We then tuned the SVM model's hyper-parameters using a grid search in order produce the best model.

In [ ]:
grid_svm = {'alpha':[],'n_iter':[],'rand_state':[] ,'Accuracy':[],'Precision':[],'Sensitivity':[],'Specificity':[]}
for alpha in [1e-14,1e-12,1e-10,1e-8,1e-6,1e-4,1e-2]:
    for n_iter in [3,8,13,18,23,28,33]:
        for rnd in [20,40,60,80]:
            SVM = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=alpha, n_iter=n_iter, random_state=rnd)),
            ])
            temp1 = test_clf(new_duplicate,SVM,None)
            grid_svm['alpha'].append(alpha)
            grid_svm['n_iter'].append(n_iter)
            grid_svm['rand_state'].append(rnd)            
            grid_svm['Accuracy'].append(temp1['Accuracy'])
            grid_svm['Precision'].append(temp1['Precision'])
            grid_svm['Sensitivity'].append(temp1['Sensitivity'])
            grid_svm['Specificity'].append(temp1['Specificity'])

In [ ]:
grid_df = pd.DataFrame(grid_svm)
grid_df['AdjustedMean'] = grid_df[['Accuracy','Specificity','Sensitivity','Precision']].mean(axis=1).values
grid_df.plot(kind='scatter',x='alpha',y='n_iter',c = 'AdjustedMean',logx=True,xlabel='alpha')

best_alpha, best_n_iter, best_rnd = grid_df.sort_values('AdjustedMean',ascending = 0)[['alpha','n_iter','rand_state']].iloc[0,:]
print {'Best Alpha':best_alpha,'Best Num Iter':best_n_iter,'Best Random State':best_rnd}

#### Running Multinomial Naive Bayes on Top Words

Next we tried running the models on just the top words of each field rather than the whole bag of words approach. Since this produces a new hyper-parameter, <tt>top_x</tt>, which dictates how many of the top words are used in the model. Using this hyper-parameter, we built ROC and Precision-Recall curves as well as plotted the accuracy for the various values of <tt>top_x</tt>. 

In [ ]:
def my_integrate(x,y,step=0.1):
    """
        Computes the area under the curve of a spline 
        interpolated from x and y over [0,1].
    """
    from scipy.interpolate import spline
    # Removes repeated x values for spline interpolation
    temp = pd.DataFrame({'x':x,'y':y,'r':[round(t,3) for t in x]}).sort_values('r').groupby('r').mean()
    xnew = [float(z)*step for z in range(0,int(float(1)/step))]
    f2 = spline(xk=temp['x'], yk=temp['y'], xnew=xnew)
    dx = [xnew[i]-xnew[i-1] for i in xrange(1,len(xnew))]
    area = sum(dx*f2[1:])
    return(area)

In [ ]:
temp = test_clf(new_duplicate,NB,topwords,15)
print {x:round(y,4) for x,y in temp.items()}

indx = range(1,40,1)
temp1 = [test_clf(new_duplicate,NB,topwords,i) for i in indx]

CM = {'Accuracy'   :[x['Accuracy']    for x in temp1],
      'Precision'  :[x['Precision']   for x in temp1],
      'Sensitivity':[x['Sensitivity'] for x in temp1],
      'Specificity':[x['Specificity'] for x in temp1],
      'Fallout'    :[x['Fallout']     for x in temp1]}

# Plot ROC Curve with y=x line for reference
_,ax = plt.subplots(1,3,figsize = [15,5])
_=pd.DataFrame(CM).plot(kind = 'scatter',y = 'Precision',x = 'Sensitivity',ax=ax[0],color = 'red',title='ROC Curve')
_=pd.DataFrame(CM).plot(kind = 'scatter',y = 'Sensitivity',x = 'Fallout',ax=ax[0],color = 'blue')
_=ax[0].legend(labels=['Prec-Recall','ROC']), ax[0].set_xlabel("Fallout/Recall"), ax[0].set_ylabel("Sensitivity/Precision")
_=ax[0].plot([0, 1], [0, 1], 'k-', lw=2)
area = 'AUC: ' + str(round(my_integrate(x=CM['Sensitivity']+CM['Fallout'],y=CM['Precision']+CM['Sensitivity'],step=0.001),3))
plt.text(0.6, 0.2,area, transform=ax[0].transAxes)

_=pd.DataFrame(CM['Accuracy']).plot(ax=ax[1],title='Accuracy vs. Top_X',color='blue')
ax[1].set_xlabel("# of Words Used"), ax[1].set_ylabel("Accuracy")

_=pd.DataFrame(CM['Accuracy']).plot(kind = 'kde',ax=ax[2],title='Accurace KDE',color='blue')
ax[2].axvline(x=np.mean(CM['Accuracy']),color='red')
ax[2].set_xlabel("Accuracy")
ax[2].legend(labels=['Dist','Mean'])


print {'Min: ' : str(min(CM['Accuracy'])),
       'Max: ' : str(max(CM['Accuracy'])),
       'Mean: ' : str(np.mean(CM['Accuracy']))}

#### Running SVM on Top Words

After determining the best values for <tt> rand_state, n_iter,</tt> and <tt> alpha </tt> we ran the tuned SVM model on just the top words of each field tuning the <tt>top_x</tt> hyper-parameter to build the ROC curve.

In [ ]:
from sklearn.linear_model import SGDClassifier
SVM = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=best_alpha, n_iter=best_n_iter, random_state=int(best_rnd))),
                ])
svm_cm = test_clf(new_duplicate,SVM,topwords,15)
print {x:round(y,4) for x,y in svm_cm.items()}

indx = range(1,40,1)
temp1 = [test_clf(new_duplicate,SVM,topwords,i) for i in indx]

CM = {'Accuracy'   :[x['Accuracy']    for x in temp1],
      'Precision'  :[x['Precision']   for x in temp1],
      'Sensitivity':[x['Sensitivity'] for x in temp1],
      'Specificity':[x['Specificity'] for x in temp1],
      'Fallout'    :[x['Fallout']     for x in temp1]}

# Plot ROC Curve with y=x line for reference
_,ax = plt.subplots(1,3,figsize = [15,5])
_=pd.DataFrame(CM).plot(kind = 'scatter',y = 'Precision',x = 'Sensitivity',ax=ax[0],color = 'red', title='ROC Curve')
_=pd.DataFrame(CM).plot(kind = 'scatter',y = 'Sensitivity',x = 'Fallout',ax=ax[0],color = 'blue')
_=ax[0].legend(labels=['Prec-Recall','ROC']), ax[0].set_xlabel("Fallout/Recall"), ax[0].set_ylabel("Sensitivity/Precision")
_=ax[0].plot([0, 1], [0, 1], 'k-', lw=2)
area = 'AUC: ' + str(round(my_integrate(x=CM['Sensitivity']+CM['Fallout'],y=CM['Precision']+CM['Sensitivity'],step=0.001),3))
plt.text(0.7, 0.2,area, transform=ax[0].transAxes)

_=pd.DataFrame(CM['Accuracy']).plot(ax=ax[1],title='Accuracy vs. Top_X',color='blue')
ax[1].set_xlabel("# of Words Used"), ax[1].set_ylabel("Accuracy")

_=pd.DataFrame(CM['Accuracy']).plot(kind = 'kde',ax=ax[2],title='Accuracy KDE',color='blue')
ax[2].axvline(x=np.mean(CM['Accuracy']),color='red')
ax[2].set_xlabel("Accuracy")
ax[2].legend(labels=['Dist','Mean'])

print {'Min: ' : str(min(CM['Accuracy'])),
       'Max: ' : str(max(CM['Accuracy'])),
       'Mean: ' : str(np.mean(CM['Accuracy']))}

Here we see that the accuracy and ROC curve are far superior with the topwords SVM model over the MNB model. This could be partly due to the fact that MNB relies on the features (words) being independent from each other which is not necesarily a good assumption. This is because a theorems choice of wording could affect all other words in the Theorem. For example, a linear algebra theorem could discuss a function as a map or an operator depending on if it is a theorem discusses the linearity of functions, a map between two vector spaces, or a Hermitian operator. In all these cases, the theorem should clearly be classified as Linear Algebra but has vastly different wording. The SVM model makes no such assumption, and it quickly weights non-informative features to zero, leaving only the big words playing a role in the classification. This is seen as the accuracy quickly rises to nearly 100% at only about 7 top words while the MNB model needs 27 to get close to the same accuracy. This is also reflected in the area under the ROC curve (AUC). For MNB this is only 0.091, significantly smaller than the ideal 1.0, while the SVM model gets much closer at 0.711. The reason for MNB's low AUC is that recall just never makes it past 20%. The recall, or true positive rate, is very lower for MNB. Since this value is computed of the recall of choosing each class as true, with the remaining set to false, this demonstrates the lack of granularity in the MNB model. For each of these recall calculations, we have only a handful true classes and mostly false classes, so the MNB model simply markes these as true instead of tweaking its weights to handle such a small number of responses. Both models had very lower Fallouts (or false positive rates), thus a misclassifed positive value was very unlikely. This is again most likely due to the 1 vs all method of multi-class classification; it was so unlikely to label anything as true, thus it was even more unlikely to label the wrong sample as true.

### The Most Beautiful Theorems

Following an article on the most "Beautiful" theorems of Mathematics from <a href=https://www.quora.com/Which-are-the-most-beautiful-mathematical-theorems-and-why target='_blank'>www.quora.com</a>, the following illustrates some traits of all these theorems as a collection. Quora asked people to consider the following when selecting theorems for the list

| Category | Description|
|----------|------------|
|_Generality_ | it is applicable to a wide variety of problems.|
|_Succinctness_|  it is expressible simply, in only a few words or equations.|
|_Originality_ | it expresses a surprising mathematical insight, or a connection between different areas of mathematics, that had not previously been widely suspected.|
|_Significance_ | it represents an important advance in mathematical knowledge, or resolves an important mathematical problem.|
|_Potency_ | it stimulates many new areas of mathematical research.|
|_Centrality_ | it is used in the proofs of many subsequent theorems.|
|_Independence_ | its proof depends on only a small number of previously established theorems, and preferably none.|

The theorems that were selected were
<center>
The Pythagorean Theoem (Geometry, Pythagoras),
Euclid's Theorem of the Infinitude of Primes (Number Theory, Euclid), The Minimax Theorem (Game Theory, John von Neumann),The Brouwer Fixed Point Theorem (Topology, Luitzen Brouwer),Cauchy's Residue Theorem (Complex Analysis, Augustin-Louis Cauchy),Fourier's Theorem (Function Theory, Joseph Fourier),The Halting Theorem (Computability Theory, Alan Turing),Gödel's Incompleteness Theorems (Mathematical logic/Metamathematics, Kurt Godel),Schubert's Prime Knot Factorization Theorem (Knot Theory, Horst Schubert),Cantor's Theorem (Set Theory/Transfinite Analysis, Georg Cantor),Fundamental Theorem of Algebra (Algebra)
</center>

For this web page, the Theorems and which category they satisfy from the above list are bolded. Thus by scraping the page for the <tt> <"b"> </tt> tag we can get the relevant information after trimming the header (first 7 lines). Moreover, to make the theorem titles match the pre-existing data theorems frame, we had to do some serious trimming. This is done be first correcting any spelling errors, i.e. theoem instead of theorem, and stripping trailing white spaces as well as 'The' from each title.

In [ ]:
## Scraping the Theorems and the considerations for each theorem
beau_url = 'https://www.quora.com/Which-are-the-most-beautiful-mathematical-theorems-and-why'
beau_soup = BeautifulSoup(urlopen(beau_url),'html.parser').find_all('b')[7:]
bolded = [x.get_text() for x in beau_soup if x.get_text()!='']
beau_thms  = [re.sub('Theorems|Theorem|Theoem','theorem', re.sub("The ",'', y)).strip() for y in bolded if re.search('theorem|theoem',y.lower())]
beau_terms = [y for y in bolded if re.search('theorem|theoem',y.lower())==None]
beau_dic = {x[0]:x[1] for x in zip(beau_thms,beau_terms)}
beau_dic

Even after trimming the titles, there were still some that were names differently than our data frame. For example, Residue theorem was called Cauchy's Residue theorem. So we had to correct these titles before searching our data frame for their text.

In [ ]:
# Trim Dictionary to match title names
beau_dic["Residue theorem"] = beau_dic.pop("Cauchy's Residue theorem")
beau_dic["Euclid's theorem"] = beau_dic.pop("Euclid's theorem of the Infinitude of Primes")
beau_dic["Brouwer fixed point theorem"] = beau_dic.pop("Brouwer Fixed Point theorem")
beau_dic[u"G\xf6del's incompleteness theorem"] = beau_dic.pop(u"G\xf6del's Incompleteness theorem")
beau_dic["Fundamental theorem of algebra"] = beau_dic.pop("theorem (Fundamental theorem of Algebra): Every polynomial [math]p : \\mathbb{C} \\rightarrow \\mathbb{C} [/math] of degree [math] n[/math] has [math] n [/math] roots (counting multiplicity) in [math] \\mathbb{C} [/math]")
beau_dic

After fixing the titles we could join this list of Beautiful Theorems with the associated "reasons" for being beautiful with our pre-existing data frame.

In [ ]:
# Build a Data frame using the beautiful theorems in duplicate_theorems
BeautifulThms = pd.concat([duplicate_theorems[duplicate_theorems['Title'] == x] for x in beau_dic.keys() if x in " ".join(duplicate_theorems['Title'])])
BeautifulThms['Reasons'] = [re.sub('\\.','',beau_dic[x]) for x in BeautifulThms['Title']]
BeautifulThms[0:3]

Then we plotted the most common words in all the Theorems that are deemed Beautiful. 

In [ ]:
# Build Wordclouds from all the Reasons
f1,axs = plt.subplots(1,2,figsize = [20,5])
for label in (axs[1].get_xticklabels() + axs[1].get_yticklabels()):
    label.set_fontsize(16)
    
texts = [str(BeautifulThms[BeautifulThms['Title']==x]['Trimmed'].values) for x in BeautifulThms['Title'].values]
temp = pd.DataFrame({'Trimmed':[remove_small([x.lower() for x in (" ".join(texts)).split() if re.search("[0-9]",str(x))==None],4)]})
txt=f(temp,axs[0],stopwords=stopwords+['which','there'],min_len=min_len,title = 'All')
_=txt[0:10].set_index('Words').plot(kind='bar',ax=axs[1],title='All',color='blue')
_=plt.gcf().subplots_adjust(bottom=0.25)

There isn't much that can be gained from these plots, but one interesting feature is that the word "proof" was by far the most common word, nearly doubling second place. This is pretty clear when you consider that a theorem must be proven, but more importantly this didn't show up in the other word clouds. Thus this likely demonstrates that there is minimal similarity between the Beautiful theorems.

In [ ]:
BeautySim = {'T1':[], 'F1':[],'Key1':[],'T2':[],'F2':[],'Key2':[],'Sim':[],'R1':[],'R2':[]}
beauty_Key = {x:field_key[i] for x,i in enumerate(BeautifulThms['Field'])}
for cmb in combos[BeautifulThms.shape[0]]:
    x,y = cmb
    BeautySim['F1'].append(BeautifulThms.Field.values[x])
    BeautySim['F2'].append(BeautifulThms.Field.values[y])
    BeautySim['T1'].append(BeautifulThms.Title.values[x])
    BeautySim['T2'].append(BeautifulThms.Title.values[y])
    BeautySim['R1'].append(BeautifulThms.Reasons.values[x])
    BeautySim['R2'].append(BeautifulThms.Reasons.values[y])
    BeautySim['Key1'].append(beauty_Key[x])
    BeautySim['Key2'].append(beauty_Key[y])
    BeautySim['Sim'].append(sim[beauty_Key[x],beauty_Key[y]])

BeautySim = pd.DataFrame(BeautySim).sort_values('Sim',ascending=0)
BeautySim[0:3]

Many of the theorems in this class are very similar, such as complex analysis, number theory, and mathematical logic. These fields are considered as part of the backbone of formal mathematics, which could explain why these were selected as "Most Beautiful". Moreover, being the backbone, and since _independence_ was a category for selection, these don't rely on pre-existing Mathematics; leaving minimal phrasing for building these theories and making them similar.

Next we looked at the most common "reasons" for beautfulness among the Theorems. We plotted two sets of plots; one for all the theorems, and one for just the most similar theorems. 

In [ ]:
# Build Wordclouds from all the Reasons
f1,axs = plt.subplots(1,2,figsize=[12,3])
temp = pd.DataFrame({'Trimmed':[str(x) for x in BeautifulThms['Reasons'].values]})
txt=f(temp,axs[0],stopwords=[],min_len=min_len,title = 'All')
_=txt.set_index('Words').plot(kind='bar',ax=axs[1],title='All')

# Build Wordclouds from just the top 10 Reasons
f1,axs = plt.subplots(1,2,figsize=[12,3])
temp = pd.DataFrame({'Trimmed':[str(x) for x in (BeautySim['R1']+','+BeautySim['R2']).values[0:5]]})
txt=f(temp,axs[0],stopwords=[],min_len=min_len,title = 'Top 5')
_=txt.set_index('Words').plot(kind='bar',ax=axs[1],title='Top 5')

There weren't many words to begin with, but there is a clear divide between the two sets as far as distributions go. For the second plot, significance started to play a much larger role than in the first. Moreover, originality fell from importance in the most similar categories. This makes sense, since if they were too original, they wouldn't be labeled as similar. 

We then used our trained SVM model to learn more about the beautiful theorems. First we checked the accuracy of the model in correctly predicting their fields, and then we checked what field of Mathematics would be predicted by combining all the beautiful theorems into one. 

In [ ]:
import sklearn

beauty_df = BeautifulThms.copy()
beauty_df['target'] = [field_key[x] for x in BeautifulThms.Field]
beauty_df.columns = ['target_names','filenames','Page','description','data','Reasons','target']

beauty_test = sklearn.datasets.base.Bunch(
                    description=beauty_df.description,
                    filenames=beauty_df.filenames,
                    target_names=beauty_df.target_names,
                    data=beauty_df.data,
                    target=beauty_df.target)

## Train on all the data
train = sklearn.datasets.base.Bunch(
                description=new_duplicate.description,
                filenames=new_duplicate.filenames,
                target_names=new_duplicate.target_names,
                data=new_duplicate.data,
                target=new_duplicate.target)

clf = SVM.fit(train.data, train.target)
predicted = clf.predict(beauty_test.data)

TP, FP, TN, FN = np.mean([performance(beauty_test.target, predicted,i) for i in set(predicted)],axis=0)
acc = np.mean(predicted == beauty_test.target)    
sens = TP/(TP+FN) if TP+FN > 0 else float('Inf')
spec = TN/(FP+TN) if FP+TN > 0 else float('Inf')
prec = TP/(TP+FP) if TP+FP > 0 else float('Inf')
cm = {'Accuracy':acc,'Sensitivity':sens,
      'Specificity':spec,'Precision':prec}
cm

Here wee see that the accuracy, precision, sensitivity, and specificity are all much less than the mean accuracy for the SVM model. This could be because there is a distinct difference between these "Beautiful" theorems and the standard formulaic theorems as a whole. Succinctness and originality were both categories in this list and both of these play against our model by not providing enough words and not providing similar words compared to other theorems in their respective fields. Moreover, theorems like the Fundamental Theorem of Algebra are about Algebra, but because these are the building blocks for Algebra all the algebraic tools built from them, so it requires the use of arithmetic and number theory in their proofs.

In [ ]:
## Classify all the Beauty Theorems into one
beauty_all = sklearn.datasets.base.Bunch(
                    description="All Beautiful",
                    filenames="",
                    target_names="All",
                    data=[" ".join(beauty_df.data)],
                    target=-1)
predicted = clf.predict(beauty_all.data)
{x:y for x,y in enumerate(fields) if x == predicted}

Finally the overarching field that most represents all the Beautiful theorems as a whole in Mathematical Logic. Fitting as all of Mathematics pulls from this field. Hence, Mathematical logic would likely be a strong contender in a more subjective classification of the most beautiful theorems. Moreover, Mathematical Logic the independence reason for selecting a Beautiful Theorem leaves mathematical logic as the most likely field to pull from as it is the foundation of all other mathematics. 

### Re-Categorizing Odd Fields

In the pre-processing step, we removed any theorems that were a part of some obscure field of Mathematics (per Wikipedia). The following will try to re-classify those Theorems into more appropriate fields so that they can be included with their fellow theorems. For example the field Quantum Theory really should be lumped in with Physics, and 'Several Complex Variables' is just Complex Analysis.

In [ ]:
removed_fields = [x for x in theorems['Field'].unique() if x not in duplicate_theorems['Field'].unique()]
removed_thms = theorems[[x in removed_fields for x in theorems['Field']]]
removed_thms[0:4]

In [ ]:
removed_df = removed_thms.copy()
removed_df.columns = ['target_names','filenames','Page','description','data']

removed_test = sklearn.datasets.base.Bunch(
                    description=removed_df.description,
                    filenames=removed_df.filenames,
                    target_names=removed_df.target_names,
                    data=removed_df.data,
                    target=removed_df.target_names)

predicted = clf.predict(removed_test.data)
removed_thms.loc[:,'Predicted'] = [fields[x] for x in predicted]
removed_thms[['Field','Predicted']][0:10]

From the table above we see that the predicted fields sometimes do not agree with what we would guess subjectively. It is not surprising that  subjects like Lie Algebra, Queuing Theory, and Mathematical Series get predicted as Abstract Algebra, Stochastic Processes, and Analysis respectively. However, the classification of fields like Axiom of Choice, Neural Networks, and Quadratic Forms as Model Theory, Partial Differential Equations, and Number Theory, didn't agree with our first guesses of their parent fields. This is a demonstration that this isn't a classification of the field itself, but the theorem that Wikipedia classified as that original field. For example the Neural Network theorem was the "Universal approximation theorem" which describes 

>"In the mathematical theory of artificial neural networks, the universal approximation theorem states[1] that a feed-forward network with a single hidden layer containing a finite number of neurons (i.e., a multilayer perceptron), can approximate continuous functions on compact subsets of $\mathbb{R}^n$, under mild assumptions on the activation function. The theorem thus states that simple neural networks can represent a wide variety of interesting functions when given appropriate parameters; however, it does not touch upon the algorithmic learnability of those parameters.

>One of the first versions of the theorem was proved by George Cybenko in 1989 for sigmoid activation functions.[2]

>Kurt Hornik showed in 1991[3] that it is not the specific choice of the activation function, but rather the multilayer feedforward architecture itself which gives neural networks the potential of being universal approximators. The output units are always assumed to be linear. For notational convenience, only the single output case will be shown. The general case can easily be deduced from the single output case."          
~<a href="https://en.wikipedia.org/wiki/Universal_approximation_theorem">Universal Approximation Theorem</a>

Now the wording of this quote starts to provide insight into why this was classifed as a PDE theorem. This provides some valuable uses for this prediction algorithm. First of all, the publisher of these Theorems only thought of the single use of the intended Theorem, and using this prediction model we can find other fields of Mathematics that these Theorems support. Moreover, if someone studies a very specific Mathematical idea, by predicting which Field this falls under, it opens up a source for other similar ideas under that field. Under the same idea, this allows publishers of Mathematics papers to consider who else might be interested. For example, the Neural Networks researcher would realize that PDE researchers might be interested and publish in an appropriate journal to reach them.

#### Thank you for exploring this Jupyter Notebook! Please contact <a href="mailto:desherman@ucdavis.edu?subject=Your%20Awesome%20MathNet%20Project">Doug Sherman</a> with any questions or comments.